In [47]:
import os
from datetime import datetime
import shutil
from glob import glob
import rioxarray as rxr
from rioxarray.exceptions import NoDataInBounds
import rasterio.features
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython import display
from rasterio.errors import NotGeoreferencedWarning
from scipy.ndimage import gaussian_filter
import importlib.util
from rioxarray.merge import merge_arrays
from scipy.optimize import minimize
from scipy.signal import find_peaks
import warnings
import torch
import pandas as pd
import pickle
import cv2
warnings.filterwarnings("ignore", category=NotGeoreferencedWarning)
np.seterr(divide='ignore', invalid='ignore')

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

def recreate_dir(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)
    return path

def load_config(path):
    spec = importlib.util.spec_from_file_location("CFG", path)
    CFG = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(CFG)
    return CFG

In [48]:
DATA_DIR = "data/rybna_202203241528"
CFG = load_config(f"{DATA_DIR}/config.py").CALIB

In [49]:
#configure logging to file
import logging
log_path = f"{DATA_DIR}/logs/calibration_{datetime.now().strftime('%d%m%Y%H%M%S')}.log"
os.makedirs(os.path.dirname(log_path), exist_ok=True)
logging.basicConfig(filename=log_path,level=logging.INFO, format='%(asctime)s %(levelname)-8s %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
logger = logging.getLogger(__name__)
logger.handlers.clear()
#logger.addHandler(logging.StreamHandler())
logger.info("Starting procedure")

In [50]:
TMP_DIR = f"{DATA_DIR}/tmp"
recreate_dir(TMP_DIR)
TIFF_DIR = f"{DATA_DIR}/tiff"
assert os.path.exists(TIFF_DIR), "tiff_dir does not exist. Please run 1_conversion.ipynb first."
GEOTIFF_OPTIM_DIR = f"{DATA_DIR}/geotiff_optim"
assert os.path.exists(GEOTIFF_OPTIM_DIR), "geotiff_dir does not exist. Please run 1_conversion.ipynb first."
TIFF_CAL_DIR = f"{DATA_DIR}/tiff_cal"
GEOTIFF_CAL_DIR = f"{DATA_DIR}/geotiff_cal"
PLOT_CLIP_DIR = f"{DATA_DIR}/plot_clip"
PLOT_CAL_DIR = f"{DATA_DIR}/plot_cal"
TEMP_OPTIM_DATASET_DIR = f"{DATA_DIR}/temp_optim_dataset"
I_CLIP_DIR = f"{TEMP_OPTIM_DATASET_DIR}/i_clip"
J_CLIP_DIR = f"{TEMP_OPTIM_DATASET_DIR}/j_clip"
CLIP_MASK_DIR = f"{TEMP_OPTIM_DATASET_DIR}/clip_mask"

In [51]:
if CFG.CACHE and os.path.exists(f"{DATA_DIR}/footprints.pkl"):
    logger.info("Loading footprints from cache")
    with open(f"{DATA_DIR}/footprints.pkl", "rb") as f:
        footprints = pickle.load(f)
else:
    logger.info("Reading footprints from geotiffs")
    geometries = []
    names = []
    for path in tqdm(glob(f"{GEOTIFF_OPTIM_DIR}/*.tiff")):
        raster = rxr.open_rasterio(path)
        footprints = rasterio.features.shapes((raster != raster.rio.nodata).values.astype(np.int16), transform=raster.rio.transform())
        footprints = [Polygon(geom["coordinates"][0]).simplify(10) for geom, colval in footprints if colval == 1]
        assert len(footprints) == 1, "More than one footprint found"
        names.append(os.path.basename(path))
        geometries.append(footprints[0])
    footprints = gpd.GeoDataFrame({"name": names, "geometry": geometries})
    #write CRS
    footprints.crs = CFG.CRS
    with open(f"{DATA_DIR}/footprints.pkl", "wb") as f:
        pickle.dump(footprints, f)

In [52]:
#erode footprints
footprints["geometry"] = footprints["geometry"].buffer(-CFG.EROSION)

In [53]:
def nan_gaussian_filter(arr, sigma):
    """Apply gaussian filter to array while ignoring nans"""
    V=arr.copy()
    V[np.isnan(arr)]=0
    VV=gaussian_filter(V,sigma=sigma)
    W=0*arr.copy()+1
    W[np.isnan(arr)]=0
    WW=gaussian_filter(W,sigma=sigma)
    Z=VV/WW
    Z[np.isnan(arr)]=np.nan
    return Z


In [54]:
if CFG.CACHE and os.path.exists(f"{TEMP_OPTIM_DATASET_DIR}/pairs.pkl"):
    logger.info("Loading cached pairs")
    with open(f"{TEMP_OPTIM_DATASET_DIR}/pairs.pkl", "rb") as f:
        pairs_i, pairs_j, pairs_area, pairs_std = pickle.load(f)
else:
    logger.info("Generating temprature global optimization dataset")
    recreate_dir(TEMP_OPTIM_DATASET_DIR)
    recreate_dir(I_CLIP_DIR)
    recreate_dir(J_CLIP_DIR)
    recreate_dir(CLIP_MASK_DIR)
    pairs_i = []
    pairs_j = []
    pairs_area = []
    pairs_std = []
    idx = 0
    for i in tqdm(range(len(footprints))):
        i_raster = rxr.open_rasterio(f"{GEOTIFF_OPTIM_DIR}/{footprints.iloc[i]['name']}", masked=True)
        for j in range(i+1, len(footprints)):
            if footprints.iloc[i].geometry.intersects(footprints.iloc[j].geometry):
                intersection = footprints.iloc[i].geometry.intersection(footprints.iloc[j].geometry)
                if intersection.area < CFG.MIN_INTERSECTION_AREA:
                    logger.info(f"i ({i}), j ({j}): intersection area too small")
                    continue
                j_raster = rxr.open_rasterio(f"{GEOTIFF_OPTIM_DIR}/{footprints.iloc[j]['name']}", masked=True)
                try:
                    i_clip = i_raster.rio.clip([intersection])
                    j_clip = j_raster.rio.clip([intersection])
                except NoDataInBounds:
                    logger.info(f"i ({i}), j ({j}): NoDataInBounds")
                    continue
                j_clip = j_clip.rio.reproject_match(i_clip)
                i_clip = i_clip.values[0]
                j_clip = j_clip.values[0]
                i_clip = nan_gaussian_filter(i_clip, sigma=CFG.GAUSS_SIGMA)
                j_clip = nan_gaussian_filter(j_clip, sigma=CFG.GAUSS_SIGMA)
                i_std = np.nanstd(i_clip)
                j_std = np.nanstd(j_clip)
                std = np.nanmean([i_std, j_std])
                if np.isnan(std):
                    logger.info(f"i ({i}), j ({j}): std is nan")
                    continue
                mask = (~np.isnan(i_clip) & ~np.isnan(j_clip)).astype(np.int16)
                i_clip[np.isnan(i_clip)] = np.nanmean(i_clip)
                j_clip[np.isnan(j_clip)] = np.nanmean(j_clip)
                i_clips = []
                j_clips = []
                masks = []
                sizes = [256, 128, 64, 32]
                for size in sizes:
                    i_clips.append(cv2.resize(i_clip, (size, size), interpolation=cv2.INTER_LINEAR))
                    j_clips.append(cv2.resize(j_clip, (size, size), interpolation=cv2.INTER_LINEAR))
                    masks.append(cv2.resize(mask, (size, size), interpolation=cv2.INTER_NEAREST))
                    #assert i_clip, j_clip, mask dont have nans
                    if np.isnan(i_clips[-1]).any():
                        logger.info(f"i ({i}), j ({j}): i_clip has nan")
                        continue
                    if np.isnan(j_clips[-1]).any():
                        logger.info(f"i ({i}), j ({j}): j_clip has nan")
                        continue
                    if np.isnan(masks[-1]).any():
                        logger.info(f"i ({i}), j ({j}): mask has nan")
                        continue
                for i_clip, j_clip, mask, size in zip(i_clips, j_clips, masks, sizes):
                    np.save(f"{I_CLIP_DIR}/{idx}_{size}.npy", i_clip)
                    np.save(f"{J_CLIP_DIR}/{idx}_{size}.npy", j_clip)
                    np.save(f"{CLIP_MASK_DIR}/{idx}_{size}.npy", mask)
                pairs_i.append(i)
                pairs_j.append(j)
                pairs_area.append(intersection.area)
                pairs_std.append(std)
                idx += 1
    pairs_i = np.array(pairs_i)
    pairs_j = np.array(pairs_j)
    pairs_area = np.array(pairs_area)
    pairs_std = np.array(pairs_std)
    #pickle dump
    with open(f"{TEMP_OPTIM_DATASET_DIR}/pairs.pkl", "wb") as f:
        pickle.dump((pairs_i, pairs_j, pairs_area, pairs_std), f)

100%|██████████| 450/450 [51:42<00:00,  6.89s/it]  


In [55]:
#pytorch dataset
class TempOptimDataset(torch.utils.data.Dataset):
    def __init__(self, i_clip_dir, j_clip_dir, pairs_i, pairs_j, pairs_area, size):
        self.i_clip_dir = i_clip_dir
        self.j_clip_dir = j_clip_dir
        self.pairs_i = pairs_i
        self.pairs_j = pairs_j
        self.pairs_area = pairs_area
        self.size = size
        assert len(self.pairs_i) == len(self.pairs_j) == len(self.pairs_area)
        
    def __len__(self):
        return len(self.pairs_i)
    def __getitem__(self, idx):
        i_clip = np.load(f"{self.i_clip_dir}/{idx}_{self.size}.npy")
        j_clip = np.load(f"{self.j_clip_dir}/{idx}_{self.size}.npy")
        mask = np.load(f"{CLIP_MASK_DIR}/{idx}_{self.size}.npy")
        i_clip = torch.tensor(i_clip, dtype=torch.float32)
        j_clip = torch.tensor(j_clip, dtype=torch.float32)
        mask = torch.tensor(mask, dtype=torch.float32)
        i_idx = torch.tensor(self.pairs_i[idx])
        j_idx = torch.tensor(self.pairs_j[idx])
        area = torch.tensor(self.pairs_area[idx])
        #resize to 256x256
        return i_idx, j_idx, i_clip, j_clip, mask, area

In [78]:
CFG = load_config(f"{DATA_DIR}/config.py").CALIB
print(CFG.BATCH_SIZE)
print(CFG.LEARNING_RATE)
i_clips = []
j_clips = []
masks = []
for i in tqdm(range(len(pairs_i)), desc="Loading clips"):
    i_clips.append(np.load(f"{I_CLIP_DIR}/{i}_{CFG.SIZE}.npy"))
    j_clips.append(np.load(f"{J_CLIP_DIR}/{i}_{CFG.SIZE}.npy"))
    masks.append(np.load(f"{CLIP_MASK_DIR}/{i}_{CFG.SIZE}.npy"))
i_clips = np.array(i_clips)
j_clips = np.array(j_clips)
masks = np.array(masks)

2048
1e-12


Loading clips: 100%|██████████| 10142/10142 [01:59<00:00, 84.54it/s]


/tmp/ipykernel_18618/351468278.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pairs_i = torch.tensor(pairs_i, dtype=torch.int64)
/tmp/ipykernel_18618/351468278.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pairs_j = torch.tensor(pairs_j, dtype=torch.int64)
/tmp/ipykernel_18618/351468278.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pairs_area = torch.tensor(pairs_area, dtype=torch.float32)
/tmp/ipykernel_18618/351468278.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detac

In [125]:
n_images = len(footprints)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pairs_i = torch.tensor(pairs_i, dtype=torch.int64)
pairs_j = torch.tensor(pairs_j, dtype=torch.int64)
pairs_area = torch.tensor(pairs_area, dtype=torch.float32)
pairs_std = torch.tensor(pairs_std, dtype=torch.float32)
i_clips = torch.tensor(i_clips, dtype=torch.float32)
j_clips = torch.tensor(j_clips, dtype=torch.float32)
masks = torch.tensor(masks, dtype=torch.float32)

a_coefs = torch.ones(n_images, dtype=torch.float32, device=device, requires_grad=True)
b_coefs = torch.zeros(n_images, dtype=torch.float32, device=device, requires_grad=True)
optimizer = torch.optim.Adam([a_coefs, b_coefs], lr=1.e-1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=25, cooldown=25, verbose=True)

best_loss = np.inf
es_counter = 0
losses = []
for epoch in (pbar := tqdm(range(CFG.EPOCHS))):
    pbar.set_description(f"Epoch {epoch}")
    optimizer.zero_grad()
    i_clips_cal = a_coefs[pairs_i, None, None] * i_clips + b_coefs[pairs_i, None, None]
    j_clips_cal = a_coefs[pairs_j, None, None] * j_clips + b_coefs[pairs_j, None, None]
    rel_loss = torch.mean((i_clips_cal - j_clips_cal)**2 * masks)
    abs_loss = 1.e-0*(torch.mean((i_clips_cal - i_clips)**2 * masks) + torch.mean((j_clips_cal - j_clips)**2 * masks))
    #rel_loss = torch.mean(torch.abs(i_clips_cal - j_clips_cal) * masks)
    #abs_loss = 1.e-6*torch.mean(torch.abs(i_clips_cal - i_clips)* masks) + torch.mean(torch.abs(j_clips_cal - j_clips)* masks)
    loss = rel_loss + abs_loss
    
    if best_loss-loss.item() > 1.e-6:
        best_loss = loss.item()
        best_a_coefs = a_coefs.detach().cpu().numpy()
        best_b_coefs = b_coefs.detach().cpu().numpy()
        es_counter = 0
    else:
        es_counter += 1
        if es_counter > 50:
            print("Early stopping")
            break
    
    loss.backward()
    optimizer.step()
    scheduler.step(loss)
    pbar.set_postfix({"rel_loss": rel_loss.item(), "abs_loss": abs_loss.item(), "loss": loss.item(), "a_mean": a_coefs.mean().item(), "b_mean": b_coefs.mean().item()})

/tmp/ipykernel_18618/914905704.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pairs_i = torch.tensor(pairs_i, dtype=torch.int64)
/tmp/ipykernel_18618/914905704.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pairs_j = torch.tensor(pairs_j, dtype=torch.int64)
/tmp/ipykernel_18618/914905704.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pairs_area = torch.tensor(pairs_area, dtype=torch.float32)
/tmp/ipykernel_18618/914905704.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detac

Epoch 00165: reducing learning rate of group 0 to 1.0000e-02.


Epoch 223:   0%|          | 223/100000 [01:01<7:22:10,  3.76it/s, rel_loss=1.04, abs_loss=0.369, loss=1.41, a_mean=0.961, b_mean=0.356]

Epoch 00223: reducing learning rate of group 0 to 1.0000e-03.


Epoch 274:   0%|          | 274/100000 [01:14<7:11:59,  3.85it/s, rel_loss=1.04, abs_loss=0.369, loss=1.41, a_mean=0.961, b_mean=0.356]

Epoch 00274: reducing learning rate of group 0 to 1.0000e-04.


Epoch 291:   0%|          | 291/100000 [01:19<7:35:36,  3.65it/s, rel_loss=1.04, abs_loss=0.369, loss=1.41, a_mean=0.961, b_mean=0.356]

Early stopping


In [127]:
recreate_dir(GEOTIFF_CAL_DIR)
# recreate_dir(TIFF_CAL_DIR)
for name, a, b in zip(tqdm(footprints["name"].values, desc="Saving calibrated rasters"), best_a_coefs, best_b_coefs):
    geotiff = rxr.open_rasterio(f"{GEOTIFF_OPTIM_DIR}/{name}", masked=True)
    geotiff.values = a * geotiff.values + b
    geotiff.rio.to_raster(f"{GEOTIFF_CAL_DIR}/{name}")
    # tiff = rxr.open_rasterio(f"{TIFF_DIR}/{name}", masked=True)
    # tiff.values = a * tiff.values + b
    # tiff.rio.to_raster(f"{TIFF_CAL_DIR}/{name}")
    # os.system(f"exiftool -tagsfromfile {TIFF_DIR}/{name} {TIFF_CAL_DIR}/{name} -overwrite_original_in_place")

Saving calibrated rasters: 100%|██████████| 450/450 [01:42<00:00,  4.39it/s]


In [ ]:


dataset = TempOptimDataset(I_CLIP_DIR, J_CLIP_DIR, pairs_i, pairs_j, pairs_area, size=CFG.SIZE)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=CFG.BATCH_SIZE, shuffle=True, num_workers=CFG.NUM_WORKERS)
n_images = len(footprints)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
a_coefs = torch.ones(n_images, dtype=torch.float32, device=device, requires_grad=True)
b_coefs = torch.zeros(n_images, dtype=torch.float32, device=device, requires_grad=True)

optimizer = torch.optim.Adam([a_coefs, b_coefs], lr=CFG.LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=5, verbose=True)
for epoch in range(CFG.EPOCHS):
    print(f"Epoch {epoch}/{CFG.EPOCHS}")
    epoch_losses = []
    for i_idx, j_idx, i_clip, j_clip, mask, area in (pbar := tqdm(dataloader)):
        optimizer.zero_grad()
        #compute loss
        i_idx, j_idx, i_clip, j_clip, area = i_idx.to(device), j_idx.to(device), i_clip.to(device), j_clip.to(device), area.to(device)
        #add singleton dimension t
        i_clip_masked = i_clip * mask
        j_clip_masked = j_clip * mask
        #fig, ax = plt.subplots(1, 2)
        #ax[0].imshow(i_clip_masked[0].detach().cpu().numpy())
        i_clip_cal = a_coefs[i_idx][:, None, None] * i_clip + b_coefs[i_idx][:, None, None]
        j_clip_cal = a_coefs[j_idx][:, None, None] * j_clip + b_coefs[j_idx][:, None, None]
        i_clip_cal_masked = i_clip_cal * mask
        j_clip_cal_masked = j_clip_cal * mask
        #ax[1].imshow(i_clip_cal_masked[0].detach().cpu().numpy())
        #plt.show()
        #diff = i_clip_cal_masked - j_clip_cal_masked
        #diff_plot = ax[2].imshow(diff[0].detach().cpu().numpy())
        #plt.colorbar(diff_plot)
        #plt.show()
        rel_loss = torch.mean(torch.abs(i_clip_cal_masked - j_clip_cal_masked))
        abs_loss = 0.0000001*(torch.mean(torch.abs(i_clip_cal_masked - i_clip_masked))+torch.mean(torch.abs(j_clip_cal_masked - j_clip_masked)))
        loss = rel_loss + abs_loss
        loss.backward()
        optimizer.step()
        #print(f"{loss.item()} = {rel_loss.item()} + {abs_loss.item()}")
        #set pbar description
        pbar.set_description(f"loss: {loss.item()}")
        epoch_losses.append(loss.item())
        
    #print random 5 a and b coefs
    print(f"Epoch loss: {np.mean(epoch_losses)}")

In [ ]:
CFG = load_config(f"{DATA_DIR}/config.py").CALIB
print(CFG.BATCH_SIZE)
print(CFG.LEARNING_RATE)
dataset = TempOptimDataset(I_CLIP_DIR, J_CLIP_DIR, pairs_i, pairs_j, pairs_area, size=CFG.SIZE)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=CFG.BATCH_SIZE, shuffle=True, num_workers=CFG.NUM_WORKERS)
n_images = len(footprints)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
a_coefs = torch.ones(n_images, dtype=torch.float32, device=device, requires_grad=True)
b_coefs = torch.zeros(n_images, dtype=torch.float32, device=device, requires_grad=True)

optimizer = torch.optim.Adam([a_coefs, b_coefs], lr=CFG.LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=5, verbose=True)
for epoch in range(CFG.EPOCHS):
    print(f"Epoch {epoch}/{CFG.EPOCHS}")
    epoch_losses = []
    for i_idx, j_idx, i_clip, j_clip, mask, area in (pbar := tqdm(dataloader)):
        optimizer.zero_grad()
        #compute loss
        i_idx, j_idx, i_clip, j_clip, area = i_idx.to(device), j_idx.to(device), i_clip.to(device), j_clip.to(device), area.to(device)
        #add singleton dimension t
        i_clip_masked = i_clip * mask
        j_clip_masked = j_clip * mask
        #fig, ax = plt.subplots(1, 2)
        #ax[0].imshow(i_clip_masked[0].detach().cpu().numpy())
        i_clip_cal = a_coefs[i_idx][:, None, None] * i_clip + b_coefs[i_idx][:, None, None]
        j_clip_cal = a_coefs[j_idx][:, None, None] * j_clip + b_coefs[j_idx][:, None, None]
        i_clip_cal_masked = i_clip_cal * mask
        j_clip_cal_masked = j_clip_cal * mask
        #ax[1].imshow(i_clip_cal_masked[0].detach().cpu().numpy())
        #plt.show()
        #diff = i_clip_cal_masked - j_clip_cal_masked
        #diff_plot = ax[2].imshow(diff[0].detach().cpu().numpy())
        #plt.colorbar(diff_plot)
        #plt.show()
        rel_loss = torch.mean(torch.abs(i_clip_cal_masked - j_clip_cal_masked))
        abs_loss = 0.0000001*(torch.mean(torch.abs(i_clip_cal_masked - i_clip_masked))+torch.mean(torch.abs(j_clip_cal_masked - j_clip_masked)))
        loss = rel_loss + abs_loss
        loss.backward()
        optimizer.step()
        #print(f"{loss.item()} = {rel_loss.item()} + {abs_loss.item()}")
        #set pbar description
        pbar.set_description(f"loss: {loss.item()}")
        epoch_losses.append(loss.item())
        
    #print random 5 a and b coefs
    print(f"Epoch loss: {np.mean(epoch_losses)}")

2048
1e-12
Epoch 0/100000


loss: 0.7762974500656128: 100%|██████████| 5/5 [02:01<00:00, 24.20s/it]


Epoch loss: 0.7754704475402832
Epoch 1/100000


loss: 0.7876381874084473: 100%|██████████| 5/5 [02:00<00:00, 24.15s/it]


Epoch loss: 0.7755790472030639
Epoch 2/100000


...

Epoch loss: 0.7756160259246826
Epoch 7/100000


loss: 0.7604162096977234: 100%|██████████| 5/5 [02:05<00:00, 25.04s/it]


Epoch loss: 0.7753184676170349
Epoch 8/100000


  0%|          | 0/5 [00:24<?, ?it/s]


KeyboardInterrupt: 

# TODO

Merge rasters

In [ ]:
def average_merge(merged_data, new_data, merged_mask, new_mask, index=None, roff=None, coff=None):
    merged_data_masked = np.ma.array(merged_data, mask=merged_mask)
    merged_data[:] = np.ma.masked_array((merged_data_masked,new_data)).mean(axis=0)
rasters = []
for path in tqdm(glob(f"{GEOTIFF_CAL_DIR}/*.tiff"), desc="Loading rasters"):
    rasters.append(rxr.open_rasterio(path, masked=True).copy())
print("Merging...")
mosaic = merge_arrays(rasters, method=average_merge)
mosaic.rio.to_raster(f"{DATA_DIR}/mosaic_cal.tiff")
print("Merging done")

Loading rasters: 100%|██████████| 77/77 [00:01<00:00, 43.92it/s]


Merging...
Merging done
